In [1]:
import pandas as pd 
from utils import read_df, save_df, sample_series
import ast
import nltk

In [2]:
df: pd.DataFrame = read_df('dados_tokenizados.csv')

In [3]:
df.sample(3)

,id,canal,texto_original,temas,texto_limpo,tokens,tokens_clean
2275,3071,Audiência Regional - Pirituba/Jaraguá,Boa noite a todos e a todas. Eu agradeço pela ...,[],boa noite a todos e a todas. eu agradeço pela ...,"['boa', 'noite', 'a', 'todos', 'e', 'a', 'toda...","['boa', 'noite', 'todos', 'todas', 'agradeco',..."
630,673,Participe+,Meta 32 - Iniciativa A - Manter - Peço atenção...,['Zeladoria Urbana e melhorias de bairro'],meta 32 - iniciativa a - manter - peço atenção...,"['meta', '32', '-', 'iniciativa', 'a', '-', 'm...","['manter', 'peco', 'atencao', 'rua', 'kenkit',..."
930,1362,Participe+,Meta 39 - Iniciativa C - Revisar - Todas as vi...,['Transportes e Mobilidade'],meta 39 - iniciativa c - revisar - todas as vi...,"['meta', '39', '-', 'iniciativa', 'c', '-', 'r...","['c', 'revisar', 'todas', 'vias', 'puderem', '..."


Para construir a bag of words vou precisar apenas dos tokens e da classificacao.

Terei que fazer o explode na classificação - pois um mesmo texto pode ter mais de um tema

In [4]:
df = df[['tokens_clean', 'temas']]

In [5]:
df.dtypes

tokens_clean    object
temas           object
dtype: object

In [6]:
def parse_lst_column(lst_str)->list[str]:

    return ast.literal_eval(lst_str)
    

In [7]:
df['temas'] = df['temas'].apply(parse_lst_column)
df['tokens_clean'] = df['tokens_clean'].apply(parse_lst_column)

In [8]:
df.sample(3)

,tokens_clean,temas
1255,"[revisar, acguarapiranga, agua, beber, ter, ze...",[Meio Ambiente]
2257,"[boa, noite, todos, todas, pessoas, prefeitura...",[]
424,"[boa, noite, todos, gostaria, agradecer, todos...",[Esporte e Lazer]


In [9]:
df = df.explode('temas')
df.rename({'temas' : 'tema'}, axis=1, inplace=True)

df.sample(3)

,tokens_clean,tema
1698,"[pontos, geracao, trabalho, renda, criar, prog...",Desenvolvimento Econômico e Trabalho
534,"[boa, noite, tudo, bem, ouvem, espero, sim, be...",Habitação
2253,"[boa, noite, todos, quero, manifestar, satisfa...",Inovação e Tecnologia


In [10]:
df.reset_index(drop=True, inplace=True)

In [11]:
df['id'] = df.index

In [12]:
df.head()

,tokens_clean,tema,id
0,"[boa, tarde, todas, todos, nome, francisco, ap...",Meio Ambiente,0
1,"[oi, mario, sergio, bortolotto, favor, respond...",Urbanismo e Licenciamento,1
2,"[entao, boa, tarde, todos, todas, comprimento,...",Zeladoria Urbana e melhorias de bairro,2
3,"[entao, boa, tarde, todos, todas, comprimento,...",Desenvolvimento Econômico e Trabalho,3
4,"[obrigada, agradeco, oportunidade, faco, parte...",Zeladoria Urbana e melhorias de bairro,4


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
df['corpora'] = df['tokens_clean'].apply(lambda x: ' '.join(x))

In [15]:
df.head()

,tokens_clean,tema,id,corpora
0,"[boa, tarde, todas, todos, nome, francisco, ap...",Meio Ambiente,0,boa tarde todas todos nome francisco apelido c...
1,"[oi, mario, sergio, bortolotto, favor, respond...",Urbanismo e Licenciamento,1,oi mario sergio bortolotto favor responde enco...
2,"[entao, boa, tarde, todos, todas, comprimento,...",Zeladoria Urbana e melhorias de bairro,2,entao boa tarde todos todas comprimento secret...
3,"[entao, boa, tarde, todos, todas, comprimento,...",Desenvolvimento Econômico e Trabalho,3,entao boa tarde todos todas comprimento secret...
4,"[obrigada, agradeco, oportunidade, faco, parte...",Zeladoria Urbana e melhorias de bairro,4,obrigada agradeco oportunidade faco parte rede...


In [16]:
df = df[['id', 'corpora', 'tema']]

In [17]:
df.head()

,id,corpora,tema
0,0,boa tarde todas todos nome francisco apelido c...,Meio Ambiente
1,1,oi mario sergio bortolotto favor responde enco...,Urbanismo e Licenciamento
2,2,entao boa tarde todos todas comprimento secret...,Zeladoria Urbana e melhorias de bairro
3,3,entao boa tarde todos todas comprimento secret...,Desenvolvimento Econômico e Trabalho
4,4,obrigada agradeco oportunidade faco parte rede...,Zeladoria Urbana e melhorias de bairro


In [18]:
vectorizer = CountVectorizer()

In [19]:
bow_matrix= vectorizer.fit_transform(df['corpora'])

In [20]:
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [21]:
bow_df.head()

,1000km,100km,10ampliar,10h,10km,119c10201210,11o,12e3,12milhoes,12o,...,zildo,zituo,zn,zogbi,zom,zona,zonas,zoneamento,zr1,zum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df.shape

(3581, 3)

In [23]:
bow_df.shape

(3581, 16701)

In [24]:
bow_df['id'] = df['id']

In [25]:
bow_df['y'] = df['tema']

In [26]:
bow_df.to_parquet(os.path.join())

'../data/bag_of_words_df.csv'